In [1]:
# !pip install datasets

In [2]:
# !pip install pandas

In [4]:
import pandas as pd
import numpy as np

In [3]:
from datasets import load_dataset
dataset = load_dataset('darrow-ai/legal-task')
dataset = dataset['train'].to_pandas()
dataset.head()

C:\Users\ronyg\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,text,domain
0,r-e4EYcBD5gMZwcz41zP,UNITED STATES DISTRICT COURT \nEASTERN DISTRIC...,consumer fraud
1,i9H5DocBD5gMZwcztj0y,IN THE UNITED STATES DISTRICT COURT \nFOR THE ...,privacy
2,SMn3DYcBD5gMZwcz-hwH,IN THE UNITED STATES DISTRICT COURT\n FOR THE ...,privacy
3,GMIWDYcBD5gMZwczDQBb,Case No. _______________ \n \n \nCLASS ACTION ...,criminal & enforcement
4,lELw_IgBF5pVm5zYONwC,UNITED STATES DISTRICT COURT \n SOUTHERN DISTR...,consumer fraud


In [4]:
dataset.id.count()

np.int64(1204)

In [5]:
dataset.groupby('domain').agg(
    total_count=('id', 'count'),
    distinct_count=('id', 'nunique')
).sort_values('distinct_count', ascending=False)


,total_count,distinct_count
domain,,
consumer fraud,200,200
securities,200,200
privacy,200,200
employment & labor,200,200
"civil rights, immigration, family",167,167
antitrust,126,126
products liability and mass tort,56,56
discrimination,20,20
criminal & enforcement,16,16


In [6]:
# print(dataset.sample(1).text.iloc[0])


# Deduplication

In [7]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", 300)
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.grid"] = True

print("Initial shape:", dataset.shape)

# Find all rows that are duplicated on the exact pair (id, text)
dup_mask = dataset.duplicated(subset=["id", "text"], keep=False)
duplicates_df = dataset[dup_mask].copy()

print(f"Number of rows involved in [id, text] duplicates: {duplicates_df.shape[0]}")

# Save them for transparency / potential manual inspection
if not duplicates_df.empty:
    duplicates_df.to_csv("duplicates_id_text.csv", index=False)
    print("Saved [id, text] duplicates to 'duplicates_id_text.csv'.")

# Now create a deduplicated version keeping the first occurrence
dataset_deduped = dataset.drop_duplicates(subset=["id", "text"], keep="first").reset_index(drop=True)

print("Shape after deduplication on [id, text]:", dataset_deduped.shape)


Initial shape: (1204, 3)
Number of rows involved in [id, text] duplicates: 0
Shape after deduplication on [id, text]: (1204, 3)


# Modeling design

## General Flow

> - We merge very sparse domains into an `"other"` class to ensure robust learning.
> - We wrap `SentenceTransformers` in a custom `SentenceTransformerFeaturizer` so it can be used
>   seamlessly inside an sklearn `Pipeline` and `RandomizedSearchCV`.
> - The embeddings are generated using **`all-MiniLM-L6-v2`**, producing dense semantic representations
>   for each case.
> - **Normalization:** during embedding generation, the vectors are **L2-normalized** directly by the
>   SentenceTransformers encoder (`normalize_embeddings=True`).
>   This ensures that each embedding has unit length, making cosine similarity equivalent to
>   the dot product and stabilizing training for linear models.
> - On top of these normalized embeddings, we train a **regularized Logistic Regression (Logit Regression)**
>   classifier — a simple yet powerful linear model that works well with dense features.
> - The model’s regularization strength (`C`) is tuned via **stratified cross-validation**
>   optimizing the **macro F1** score, which gives equal weight to all classes despite imbalance.

### Adapting SentenceTransformers to scikit-learn

> By default, **SentenceTransformers** models (e.g. `all-MiniLM-L6-v2`) operate outside of the
standard scikit-learn API — they take a list of texts and return NumPy embeddings, not a
DataFrame or sparse matrix.
>
> To integrate them cleanly with scikit-learn’s ecosystem (pipelines, cross-validation,
hyper-parameter search), we wrap the embedding step inside a **custom transformer**
that follows the `fit` / `transform` interface.

#### Key design ideas

1. **Custom Transformer class**
   We implement a small class called `SentenceTransformerFeaturizer`
   that inherits from `BaseEstimator` and `TransformerMixin`.
   - `fit()` simply loads or initializes the embedding model.
   - `transform()` encodes the input texts into dense vectors using
     `SentenceTransformer.encode()`.

2. **Sklearn compatibility**
   Because the class exposes `.fit()` and `.transform()`, it can be inserted as the first
   stage in an `sklearn.Pipeline`:
   ```python
   pipe = Pipeline([
       ("embed", SentenceTransformerFeaturizer(model_name="all-MiniLM-L6-v2")),
       ("clf", LogisticRegression(...))
   ])



In [9]:
import re
import numpy as np

# Start from your deduped / filtered dataframe:
# df has columns: id, text, domain

def clean_case_text(text: str) -> str:
    """
    Light normalization for semantic similarity:
    - ensure string
    - normalize whitespace (incl. newlines) to single spaces
    - strip
    - lowercase
    We keep punctuation and numbers – they can be informative in legal texts.
    """
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    # Collapse all whitespace (spaces, tabs, newlines) into a single space
    text = re.sub(r"\s+", " ", text)

    # Strip leading/trailing spaces
    text = text.strip()

    # Lowercase for stability
    text = text.lower()

    return text

# Apply cleaning
dataset_deduped["text_clean"] = dataset_deduped["text"].apply(clean_case_text)

# Build helper mappings
case_ids = dataset_deduped["id"].tolist()
id_to_index = {cid: i for i, cid in enumerate(case_ids)}


In [ ]:
import os
import json
import re
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score,
)
from sklearn.preprocessing import label_binarize
from scipy.stats import loguniform
import matplotlib.pyplot as plt
import joblib

# ============================================
# Global config
# ============================================

OUTPUT_DIR = "outputs"


def _ensure_output_dir():
    os.makedirs(OUTPUT_DIR, exist_ok=True)


# ============================================
# 1. Rare domains pooling
# ============================================

RARE_TO_OTHER = {
    "healthcare",
    "intellectual property & communication",
    "environmental & natural resources",
}


def _merge_rare_domains(y: pd.Series) -> pd.Series:
    """
    Map very sparse domains into a single 'other' class.
    """
    return y.apply(lambda label: "other" if label in RARE_TO_OTHER else label)


# ============================================
# 2. Simple SentenceTransformer featurizer
# ============================================

class SentenceTransformerFeaturizer(BaseEstimator, TransformerMixin):
    """
    Sklearn transformer:
    Expects a DataFrame with 'text_clean' column
    -> returns sentence-transformer embeddings.
    """

    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", text_col="text_clean"):
        self.model_name = model_name
        self.text_col = text_col
        self.model = None

    def fit(self, X, y=None):
        # Load the sentence-transformers model once
        from sentence_transformers import SentenceTransformer
        self.model = SentenceTransformer(self.model_name)
        return self

    def transform(self, X):
        # Support both DataFrame with text_col, or direct iterable of texts
        if isinstance(X, pd.DataFrame):
            texts = X[self.text_col].astype(str).tolist()
        else:
            texts = pd.Series(X).astype(str).tolist()

        embeddings = self.model.encode(
            texts,
            batch_size=32,
            show_progress_bar=False,
            normalize_embeddings=True,
        )
        return np.asarray(embeddings)


# ============================================
# 3. Training function
# ============================================

def train_domain_model(X, y):
    '''
    Train a text -> domain classifier with:
    - rare domain pooling
    - sentence-transformer embeddings
    - random search on LogisticRegression(C)
    - save best model in outputs/

    @param X - pandas dataframe containing all the X values (must include 'text_clean')
    @param y - pandas series containing all the y values (original domains)
    @returns a trained model (Pipeline)
    @rtype Pipeline
    '''
    _ensure_output_dir()

    # --- 3.1 Merge rare domains for training labels ---
    y_merged = _merge_rare_domains(y)

    # --- 3.2 Define pipeline: embeddings -> logistic regression ---
    pipeline = Pipeline([
        ("embed", SentenceTransformerFeaturizer()),
        ("clf", LogisticRegression(
            max_iter=200,
            multi_class="auto",
            solver="lbfgs",
        )),
    ])

    # --- 3.3 Hyperparameter search space (on C only; embeddings are fixed) ---
    param_distributions = {
        "clf__C": loguniform(1e-2, 1e2),
    }

    # --- 3.4 Stratified CV for random search ---
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=10,
        scoring="f1_macro",
        cv=cv,
        n_jobs=-1,
        verbose=1,
        random_state=42,
        refit=True,  # retrain best pipeline on all training data
    )

    # --- 3.5 Fit search on given TRAIN data ---
    search.fit(X, y_merged)

    # --- 3.6 Extract best model & save it ---
    best_model = search.best_estimator_

    model_path = os.path.join(OUTPUT_DIR, "best_domain_model.joblib")
    joblib.dump(best_model, model_path)

    # (Optional but nice) Save best params
    best_params_path = os.path.join(OUTPUT_DIR, "best_domain_model_params.json")
    with open(best_params_path, "w") as f:
        json.dump(search.best_params_, f, indent=2)

    return best_model


# ============================================
# 4. Evaluation function
# ============================================

def test_domain_model(model, X, y):
    '''
    Evaluate a trained model on a TEST set.

    - Apply same rare-domain pooling on y
    - Compute global & per-class metrics
    - Compute micro-average ROC AUC & PR AUC
    - Save metrics and ROC/PR plot into outputs/

    @param model - Pipeline object model
    @param X - pandas dataframe containing all the X test values
    @param y - pandas series containing all the y true values
    @returns a dictionary with model measures
    @rtype dict
    '''
    _ensure_output_dir()

    # --- 4.1 Align labels with training logic ---
    y_true = _merge_rare_domains(y)

    # --- 4.2 Predictions ---
    y_pred = model.predict(X)

    # --- 4.3 Global metrics ---
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average="macro")
    f1_weighted = f1_score(y_true, y_pred, average="weighted")

    # --- 4.4 Per-class metrics ---
    report = classification_report(y_true, y_pred, output_dict=True)

    # ============================================
    # 4.5 ROC & PR curves (micro-average, global summary)
    # ============================================

    roc_auc_micro = None
    pr_auc_micro = None

    # Only if model supports predict_proba (LogReg does)
    if hasattr(model, "predict_proba") or hasattr(model.named_steps.get("clf", None), "predict_proba"):
        # In a Pipeline, predict_proba is exposed at top-level if last step has it.
        y_proba = model.predict_proba(X)

        classes = model.classes_
        y_true_bin = label_binarize(y_true, classes=classes)

        # Micro-avg ROC
        fpr, tpr, _ = roc_curve(y_true_bin.ravel(), y_proba.ravel())
        roc_auc_micro = auc(fpr, tpr)

        # Micro-avg PR
        precision, recall, _ = precision_recall_curve(y_true_bin.ravel(), y_proba.ravel())
        pr_auc_micro = average_precision_score(y_true_bin, y_proba, average="micro")

        # --- Plot side-by-side ROC & PR ---
        fig, axes = plt.subplots(1, 2, figsize=(10, 4))

        # ROC curve
        axes[0].plot(fpr, tpr, label=f"Micro-avg ROC (AUC = {roc_auc_micro:.3f})")
        axes[0].plot([0, 1], [0, 1], linestyle="--", label="Random")
        axes[0].set_xlabel("False Positive Rate")
        axes[0].set_ylabel("True Positive Rate")
        axes[0].set_title("ROC Curve (micro-average)")
        axes[0].legend(loc="lower right")

        # PR curve
        axes[1].plot(recall, precision, label=f"Micro-avg PR (AP = {pr_auc_micro:.3f})")
        axes[1].set_xlabel("Recall")
        axes[1].set_ylabel("Precision")
        axes[1].set_title("Precision-Recall Curve (micro-average)")
        axes[1].legend(loc="lower left")

        plt.tight_layout()
        fig_path = os.path.join(OUTPUT_DIR, "domain_model_roc_pr_curves.png")
        plt.savefig(fig_path, dpi=150)
        plt.close(fig)

    # ============================================
    # 4.6 Save metrics to JSON
    # ============================================

    metrics = {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "roc_auc_micro": roc_auc_micro,
        "pr_auc_micro": pr_auc_micro,
        "per_class_report": report,
    }

    metrics_path = os.path.join(OUTPUT_DIR, "domain_model_metrics.json")
    with open(metrics_path, "w") as f:
        json.dump(metrics, f, indent=2, default=float)

    return metrics


In [18]:

# ============================================
# 5. Predict single sample by case_id
# ============================================

def predict_sample(model, case_id, data):
    '''
    This method should return a "domain" prediction for this sample.

    @param model - Pipeline object model
    @param case_id - the ID of the case we need to predict
    @param data - dataframe containing at least ['case_id', 'text_clean']
    @returns the predicted domain name
    @rtype str
    '''
    row = data[data["id"] == case_id]

    if row.empty:
        raise ValueError(f"case_id {case_id} not found in data")

    pred = model.predict(row)[0]
    return pred


# Application

In [ ]:
from sklearn.model_selection import train_test_split

# Suppose df has: ['case_id', 'text_clean', 'domain']
df = dataset_deduped.copy()

# 1) Stratified split on pooled labels
y_all = df["domain"]
X_all = df[["id", "text_clean"]]

X_train, X_test, y_train, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    random_state=42,
    stratify=_merge_rare_domains(y_all)
)

# 2) Train on train split
model = train_domain_model(X_train, y_train)

# 3) Evaluate on test split
metrics = test_domain_model(model, X_test, y_test)
print(metrics["accuracy"], metrics["f1_macro"])
# print(metrics["per_class_report"])  # if you want details

In [19]:

# 4) Predict one sample
some_id = X_test["id"].iloc[0]
print(predict_sample(model, some_id, df))


civil rights, immigration, family


In [ ]:
# Results

#Insights and Conclusions

## Model Overview
The final model combines **SentenceTransformers (`all-MiniLM-L6-v2`) embeddings** with a
**regularized Logistic Regression (Logit Regression)** classifier.
The embeddings are L2-normalized to preserve cosine geometry, and the model was tuned
via **RandomizedSearchCV (5-fold, macro-F1 optimization)** to balance performance across domains.

---

## 🔹 Global Performance
| Metric | Score |
|:--|--:|
| **Accuracy** | 0.718 |
| **Macro F1** | 0.513 |
| **ROC-AUC (micro)** | 0.942 |
| **PR-AUC (micro)** | 0.786 |

- The high **ROC-AUC** indicates strong global ranking ability (the model distinguishes positive vs. negative cases well overall).
- **PR-AUC** shows reasonable precision–recall trade-off given class imbalance.
- The **macro F1 ≈ 0.51** reflects uneven per-class performance — expected due to sparsely represented domains.

---

## 🔹 Per-Domain Observations
| Domain | F1-Score | Precision | Recall | Support | Key Notes |
|:--|--:|--:|--:|--:|:--|
| **Privacy** | **0.86** | 0.82 | 0.90 | 40 | Very strong semantic signal; model captures topical cues well. |
| **Employment & Labor** | **0.85** | 0.85 | 0.85 | 40 | Consistent and robust category. |
| **Civil Rights / Immigration / Family** | **0.83** | 0.87 | 0.79 | 34 | High precision and recall — good generalization. |
| **Consumer Fraud / Securities** | 0.68–0.70 | – | – | 40 each | Moderate performance; legal and financial language overlap causes confusion. |
| **Antitrust** | 0.63 | 0.62 | 0.64 | 25 | Acceptable given limited data. |
| **Products Liability / Other / Discrimination / Criminal & Enforcement** | ≤ 0.40 | – | – | < 15 each | Poor recall — small sample size and semantic overlap with major domains. |

- Classes with fewer than ~15 examples underperform due to **data scarcity** and limited representation during training.
- “Other” and “Discrimination” domains likely require either more labeled data or hierarchical grouping.

---

## 🔹 Visual Diagnostics
![Domain Model ROC & PR Curves](notebooks/outputs/domain_model_roc_pr_curves.png)

- **ROC curves** confirm strong separability for dominant classes (privacy, employment).
- **PR curves** reveal precision drop-off for minority classes, reinforcing the need for
  additional samples or class-balancing techniques.

---

##Key Takeaways
- The combination of **semantic embeddings + logit regression** provides a solid, interpretable baseline.
- Strong results in well-represented domains validate that embeddings capture legal semantics effectively.
- Performance drops for rare classes suggest:
  - Expanding labeled data or synthetic augmentation,
  - Trying hierarchical or multi-task classification,
  - Exploring domain-specific legal embeddings (e.g., `legal-bert`).

> **Overall:** the model generalizes well across main legal topics and forms a robust foundation
for subsequent fine-tuning or domain-adapted improvements.
